# Cross Validation

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = False
c.wandb.group = "LB"
c.wandb.dir = "../../cache/"
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretraind_dir = "../../datasets/trainings"

In [4]:
pretrained = f"""
- dir: {pretraind_dir}/2022-02-12_20-21-03/fold0/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-05/fold1/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-07/fold2/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-09/fold3/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-12/fold4/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-16/fold6/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-12_20-21-19/fold7/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-10/fold8/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-13/fold9/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-15/fold10/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-17/fold11/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-19/fold12/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-21/fold13/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-33-24/fold14/
  model: ump_1dcnn
- dir: {pretraind_dir}/2022-02-13_07-34-20/fold5/
  model: ump_1dcnn
"""

_pretrained = f"""
"""

c.params.pretrained = OmegaConf.create(pretrained)

In [5]:
log.info(OmegaConf.to_yaml(c))

2022-02-13 20:21:20,820 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: true
  entity: imokuri
  project: ump
  dir: ../../cache/
  group: LB
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ${settings.dirs.input}preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 440
  n_class: 1
  preprocess: []
  n_fold: 5
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_name: investment_id
  time_name: time_id
  label_name: target
  use_feature: true
  feature_set:
  - f000
  

In [6]:
import os

import pandas as pd
import src.utils as utils
from src.get_score import record_result
from tqdm.notebook import tqdm

In [7]:
run = utils.setup_wandb(c)

2022-02-13 20:21:22,465 [ERROR] [jupyter] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)


2022-02-13 20:21:26,136 [INFO] [utils] WandB initialized. name: lemon-paper-174, id: 17escr50


In [8]:
train = pd.read_feather("../../inputs/train.f")
train = train.loc[:, ["row_id", "time_id", "target"]]
train.set_index("row_id", inplace=True)

In [9]:
preds_col = []

# 各 OOF の結果を読み取り
for n, training in tqdm(enumerate(c.params.pretrained), total=len(c.params.pretrained)):
    preds_col.append(f"preds{n}")
    oof_df = pd.read_feather(os.path.join(training.dir.rsplit("/", 2)[0], "oof_df.f")).set_index("row_id")

    if training.model == "lightgbm":
        train[f"preds{n}"] = oof_df["preds"].groupby("row_id").sum()
    else:
        train[f"preds{n}"] = oof_df["preds"]

# 各行の OOF の結果の数をカウントする
train["count_oof"] = len(c.params.pretrained) - train.isnull().sum(axis=1)  # + 4

# OOF の結果がない行を 0 埋め
train.fillna(0, inplace=True)

# OOF の結果をマージ
train["preds"] = 0
for col in preds_col:
    train["preds"] += train[col]

# 推論結果がないものは除外
train = train[train["preds"] != 0.0]

# 複数 OOF を加算している場合は、OOFの数で割る
train["preds"] = train["preds"] / train["count_oof"]

  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
train

,time_id,target,preds0,preds1,preds2,preds3,preds4,preds5,preds6,preds7,preds8,preds9,preds10,preds11,preds12,preds13,preds14,count_oof,preds
row_id,,,,,,,,,,,,,,,,,,,
0_1,0,-0.300875,0.150635,0.117004,0.163330,0.101990,0.017242,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,5,0.110040
0_2,0,-0.231040,-0.004547,0.026566,0.022003,-0.026306,-0.022369,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,5,-0.000931
0_6,0,0.568807,0.067932,0.089111,0.106750,0.159546,0.046600,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,5,0.093988
0_7,0,-1.064780,-0.161377,-0.162476,-0.122009,-0.115173,-0.125732,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,5,-0.137354
0_8,0,-0.531940,-0.228638,-0.130737,-0.143188,-0.380615,-0.053284,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,5,-0.187292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219_3768,1219,0.033600,0.000000,0.000000,0.000000,0.000000,-0.063477,0.0,0.0,-0.078430,0.0,0.0,-0.063416,0.0,-0.079956,-0.037720,0.0,5,-0.064600
1219_3769,1219,-0.223264,0.000000,0.000000,0.000000,0.000000,0.139038,0.0,0.0,0.153198,0.0,0.0,0.189331,0.0,0.064026,0.141968,0.0,5,0.137512
1219_3770,1219,-0.559415,0.000000,0.000000,0.000000,0.000000,0.014793,0.0,0.0,0.190918,0.0,0.0,0.145508,0.0,0.131714,0.044586,0.0,5,0.105504


In [11]:
train["count_oof"].value_counts()

5    3141399
Name: count_oof, dtype: int64

In [12]:
record_result(c, train, c.params.n_fold)

2022-02-13 20:21:49,295 [INFO] [get_score] Score: 0.17564


0.17564203593411276